In [1]:
import numpy as np
import os
import sys
import json
import tqdm
import cutting_funcitons as cf

In [2]:
with open("./variable_length/category_file.json", 'r') as file:
    data = json.load(file)

mg_data = data[0]
o_data = data[1]
print("a0000976afc6b723c88d004aa54d4fd9" in mg_data['test'])

True


In [3]:
for h in os.listdir("./variable_length/train/complete"):
    name = h.split('.')[0]
    if name in mg_data['train']:
        os.rename(f"./variable_length/train/complete/{h}", f"./variable_length/22Mg/{h}")
    elif name in o_data['train']:
        os.rename(f"./variable_length/train/complete/{h}", f"./variable_length/16O/{h}")
    else:
        raise NameError("Hash not found in category file")
    
for h in os.listdir("./variable_length/val/complete"):
    name = h.split('.')[0]
    if name in mg_data['val']:
        os.rename(f"./variable_length/val/complete/{h}", f"./variable_length/22Mg/{h}")
    elif name in o_data['val']:
        os.rename(f"./variable_length/val/complete/{h}", f"./variable_length/16O/{h}")
    else:
        raise NameError("Hash not found in category file")
    
for h in os.listdir("./variable_length/test/complete"):
    name = h.split('.')[0]
    if name in mg_data['test']:
        os.rename(f"./variable_length/test/complete/{h}", f"./variable_length/22Mg/{h}")
    elif name in o_data['test']:
        os.rename(f"./variable_length/test/complete/{h}", f"./variable_length/16O/{h}")
    else:
        raise NameError("Hash not found in category file")

In [3]:
OUTPUT_SIZE = 2048
RNG = np.random.default_rng()

for c in tqdm.tqdm(os.listdir("./variable_length/22Mg")):
    complete = np.load(f"./variable_length/22Mg/{c}")

    sampled = np.ndarray((OUTPUT_SIZE, 4))
    og_length = len(complete)

    if og_length < OUTPUT_SIZE:
        # Up sample
        sampled[:og_length] = complete
        l = og_length
        while l < OUTPUT_SIZE:
            chosen = RNG.choice(complete)
            sampled[l] = chosen
            l += 1
        pass
    elif og_length > OUTPUT_SIZE:
        # Down sample
        sampled = complete
        RNG.shuffle(sampled, axis=0)
        pass
    else:
        # No sampling
        sampled = complete
    
    if c.split('.')[0] in mg_data['train']:
        np.save(f"./variable_length/train/complete/{c}", sampled)
    elif c.split('.')[0] in mg_data['val']:
        np.save(f"./variable_length/val/complete/{c}", sampled)
    elif c.split('.')[0] in mg_data['test']:
        np.save(f"./variable_length/test/complete/{c}", sampled)
    else:
        raise NameError(f"Hash not found in category file: {c.split('.')[0]}")
    
for c in tqdm.tqdm(os.listdir("./variable_length/16O")):
    complete = np.load(f"./variable_length/16O/{c}")

    sampled = np.ndarray((OUTPUT_SIZE, 4))
    og_length = len(complete)

    if og_length < OUTPUT_SIZE:
        # Up sample
        sampled[:og_length] = complete
        l = og_length
        while l < OUTPUT_SIZE:
            chosen = RNG.choice(complete)
            sampled[l] = chosen
            l += 1
        pass
    elif og_length > OUTPUT_SIZE:
        # Down sample
        sampled = complete
        RNG.shuffle(sampled, axis=0)
        pass
    else:
        # No sampling
        sampled = complete
    
    if c.split('.')[0] in o_data['train']:
        np.save(f"./variable_length/train/complete/{c}", sampled)
    elif c.split('.')[0] in o_data['val']:
        np.save(f"./variable_length/val/complete/{c}", sampled)
    elif c.split('.')[0] in o_data['test']:
        np.save(f"./variable_length/test/complete/{c}", sampled)
    else:
        raise NameError(f"Hash not found in category file: {c.split('.')[0]}")

100%|██████████| 3303/3303 [01:58<00:00, 27.84it/s]


In [7]:
RNG = np.random.default_rng()

for c in tqdm.tqdm(os.listdir("./variable_length/22Mg")):
    compelte = np.load(f"./variable_length/22Mg/{c}")
    num_cut = len(compelte) // 4
    num_left = len(compelte) - num_cut

    center = np.ndarray((OUTPUT_SIZE, 4))
    rand = np.ndarray((OUTPUT_SIZE, 4))
    down = np.ndarray((OUTPUT_SIZE, 4))

    center[:num_left] = cf.center_cut(compelte, (0,0), num_cut)
    rand[:num_left] = cf.rand_cut(compelte, num_cut, RNG)
    down[:num_left] = cf.down_sample(compelte, num_cut, RNG)

    for i in range(num_left, OUTPUT_SIZE):
        center[i] = RNG.choice(center[:num_left])
        rand[i] = RNG.choice(rand[:num_left])
        down[i] = RNG.choice(down[:num_left])

    if c.split('.')[0] in mg_data['train']:
        if not os.path.exists(f"./variable_length/train/partial/{c.split('.')[0]}"):
            os.mkdir(f"./variable_length/train/partial/{c.split('.')[0]}")

        np.save(f"./variable_length/train/partial/{c.split('.')[0]}/center.npy", center)
        np.save(f"./variable_length/train/partial/{c.split('.')[0]}/rand.npy", rand)
        np.save(f"./variable_length/train/partial/{c.split('.')[0]}/down.npy", down)

    elif c.split('.')[0] in mg_data['val']:
        if not os.path.exists(f"./variable_length/val/partial/{c.split('.')[0]}"):
            os.mkdir(f"./variable_length/val/partial/{c.split('.')[0]}")

        np.save(f"./variable_length/val/partial/{c.split('.')[0]}/center.npy", center)
        np.save(f"./variable_length/val/partial/{c.split('.')[0]}/rand.npy", rand)
        np.save(f"./variable_length/val/partial/{c.split('.')[0]}/down.npy", down)

    elif c.split('.')[0] in mg_data['test']:
        if not os.path.exists(f"./variable_length/test/partial/{c.split('.')[0]}"):
            os.mkdir(f"./variable_length/test/partial/{c.split('.')[0]}")

        np.save(f"./variable_length/test/partial/{c.split('.')[0]}/center.npy", center)
        np.save(f"./variable_length/test/partial/{c.split('.')[0]}/rand.npy", rand)
        np.save(f"./variable_length/test/partial/{c.split('.')[0]}/down.npy", down)

    else:
        raise NameError(f"Hash not found in category file: {c.split('.')[0]}")


for c in tqdm.tqdm(os.listdir("./variable_length/16O")):
    compelte = np.load(f"./variable_length/16O/{c}")
    num_cut = len(compelte) // 4
    num_left = len(compelte) - num_cut

    center = np.ndarray((OUTPUT_SIZE, 4))
    rand = np.ndarray((OUTPUT_SIZE, 4))
    down = np.ndarray((OUTPUT_SIZE, 4))

    center[:num_left] = cf.center_cut(compelte, (0,0), num_cut)
    rand[:num_left] = cf.rand_cut(compelte, num_cut, RNG)
    down[:num_left] = cf.down_sample(compelte, num_cut, RNG)

    for i in range(num_left, OUTPUT_SIZE):
        center[i] = RNG.choice(center[:num_left])
        rand[i] = RNG.choice(rand[:num_left])
        down[i] = RNG.choice(down[:num_left])

    if c.split('.')[0] in o_data['train']:
        if not os.path.exists(f"./variable_length/train/partial/{c.split('.')[0]}"):
            os.mkdir(f"./variable_length/train/partial/{c.split('.')[0]}")

        np.save(f"./variable_length/train/partial/{c.split('.')[0]}/center.npy", center)
        np.save(f"./variable_length/train/partial/{c.split('.')[0]}/rand.npy", rand)
        np.save(f"./variable_length/train/partial/{c.split('.')[0]}/down.npy", down)

    elif c.split('.')[0] in o_data['val']:
        if not os.path.exists(f"./variable_length/val/partial/{c.split('.')[0]}"):
            os.mkdir(f"./variable_length/val/partial/{c.split('.')[0]}")

        np.save(f"./variable_length/val/partial/{c.split('.')[0]}/center.npy", center)
        np.save(f"./variable_length/val/partial/{c.split('.')[0]}/rand.npy", rand)
        np.save(f"./variable_length/val/partial/{c.split('.')[0]}/down.npy", down)

    elif c.split('.')[0] in o_data['test']:
        if not os.path.exists(f"./variable_length/test/partial/{c.split('.')[0]}"):
            os.mkdir(f"./variable_length/test/partial/{c.split('.')[0]}")

        np.save(f"./variable_length/test/partial/{c.split('.')[0]}/center.npy", center)
        np.save(f"./variable_length/test/partial/{c.split('.')[0]}/rand.npy", rand)
        np.save(f"./variable_length/test/partial/{c.split('.')[0]}/down.npy", down)

    else:
        raise NameError(f"Hash not found in category file: {c.split('.')[0]}")


100%|██████████| 3303/3303 [05:39<00:00,  9.74it/s]
